In [1]:
#! evaluation depend on ragas and another stuffb
# !pip install pandas
# !pip install matplotlib


In [2]:
# Modules to Import

import pandas as pd
from datasets import Dataset, Features, Sequence, Value
import ast
#! ragas stuff
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["OPENAI_API_KEY"] = ""
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision
)


/home/ducdeptrai/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# temp = pd.read_csv("../Chatbot/csvGenerateQuestion.csv", encoding="utf-8")
# contextList = temp['contexts']
# contextList[0]

In [4]:
#! if user use csv file:
def CSVpreprocessing(csvPath: str):
    df = pd.read_csv(csvPath)
    questionList = df['question']
    answerList = df['answer']
    contextList = df['contexts']
    contextListConvertedEleToList = [[context] for context in contextList]
    return questionList, answerList, contextListConvertedEleToList, contextList
questionList, answerList, contextListConvertedEleToList,contextList = CSVpreprocessing("../Chatbot/csvGenerateQuestion.csv")



In [5]:
# i= 19
# print("This is my question: ", questionList[i])
# print("And this is my context: ", contextListConvertedEleToList[i])
# print("I want you to generate answer base on this context. I will use your answer for groundtruth, and the answer should good so my ragas score can achieve above 80% all score. Answer in 1 paragraph long 40 words")

#! after that, write groundtruth to csv file!
with open('./chatgptGroundTruth.txt', 'r') as file:
    a = file.readlines()
groundTruthList = a


In [6]:
df = pd.DataFrame({"question": questionList, "answer": answerList, "contexts": contextList, "groundtruth": groundTruthList})
df.to_csv("./Testdata.csv", index=False)

In [7]:

def evaluateFunction():

    data = {
        "question": questionList, #! replace with question list!
        "answer": answerList,  #! replace with response from chatbot
        "contexts": contextListConvertedEleToList,  
        "ground_truth": groundTruthList #! replace with response from EvaluateChatbot
    }

    # Define the features explicitly to ensure correct data types
    features = Features({
        "question": Value("string"),
        "answer": Value("string"),
        "contexts": Sequence(Value("string")),  # Ensuring contexts is treated as a sequence of strings
        "ground_truth": Value("string")
    })

    # Convert the dictionary to a Dataset with the specified features
    dataset = Dataset.from_dict(data, features=features)

    # Perform the evaluation using the adjusted dataset
    result = evaluate(
        dataset,
        metrics=[
            context_precision,
            faithfulness,
            answer_relevancy,
            context_recall
        ],
    )

    print(result)



In [8]:
evaluateFunction()

Evaluating:   0%|          | 0/80 [00:04<?, ?it/s]


KeyboardInterrupt: 